In [29]:
from __future__ import annotations
import io, json, os, re, time, locale
from collections import defaultdict, deque
from contextlib import contextmanager
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Tuple

import numpy as np, calendar
import pandas as pd
import requests
from requests import post
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm

import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from decimal import Decimal, ROUND_HALF_UP, getcontext
import pyautogui as pag
from time import sleep
from IPython.display import display
import ipywidgets as widgets

In [30]:
# ==============================
# cliente API Radar
# ==============================
getcontext().prec = 28  # precisão segura

def dec_ptbr(s: str) -> Decimal:
    # remove milhar e troca vírgula por ponto
    return Decimal(s.replace('.', '').replace(',', '.'))

def fmt_ptbr(d: Decimal, casas: int) -> str:
    q = Decimal('1').scaleb(-casas)  # 10^-casas
    d = d.quantize(q, rounding=ROUND_HALF_UP)
    # sempre devolve string com casas fixas (mantém zeros finais)
    return f"{d:.{casas}f}".replace('.', ',')

def casas_decimais_ptbr(s: str, padrao: int = 8) -> int:
    return len(s.split(',')[1]) if ',' in s else padrao

def read_config_template(path: str, encoding="cp1252")->str:
    with open(path, "r", encoding=encoding, errors="replace") as f:
        return f.read().replace("\n","")

def inject_dates(config: str, dt_ini: pd.Timestamp, dt_fim: pd.Timestamp)->str:
    a = pd.Timestamp(dt_ini).strftime("%d/%m/%y")
    b = pd.Timestamp(dt_fim).strftime("%d/%m/%y")
    A = pd.Timestamp(dt_ini).strftime("%d/%m/%Y")
    B = pd.Timestamp(dt_fim).strftime("%d/%m/%Y")
    config = re.sub(r'"Inicial"="[^"]*";',     f'"Inicial"="{a}";',     config)
    config = re.sub(r'"Final"="[^"]*";',       f'"Final"="{b}";',       config)
    config = re.sub(r'"DataInicial"="[^"]*";', f'"DataInicial"="{a}";', config)
    config = re.sub(r'"DataFinal"="[^"]*";',   f'"DataFinal"="{b}";',   config)
    return config

def _maybe_csv_df(obj: dict)->pd.DataFrame:
    for k in ("Arquivo","Conteudo","ConteudoArquivo","Csv","CSVText"):
        v = obj.get(k)
        if isinstance(v,str) and (";" in v or "\n" in v):
            return pd.read_csv(io.StringIO(v), sep=";", encoding="cp1252", engine="python", on_bad_lines="skip")
    return pd.DataFrame()

def json_to_dataframe(payload)->pd.DataFrame:
    if isinstance(payload, list): return pd.DataFrame(payload)
    if isinstance(payload, dict):
        csv = _maybe_csv_df(payload)
        if not csv.empty: return csv
        for k in ["Registros","results","Result","Items","rows","Data","Table","Lista","ListaRegistros","value","d"]:
            v = payload.get(k)
            if isinstance(v, list) and (len(v)==0 or isinstance(v[0], dict)):
                return pd.DataFrame(v)
            if isinstance(v, dict):
                inner = json_to_dataframe(v)
                if not inner.empty: return inner
        return pd.json_normalize(payload, max_level=1)
    return pd.DataFrame()

def radar_post_json(url: str, login_info: dict, config_string: str, timeout=(5,240), max_retries=3)->pd.DataFrame:
    headers={"Content-Type":"application/json","Accept":"application/json"}
    last=None
    for att in range(max_retries):
        try:
            r = requests.post(url, json={"login":login_info,"config":config_string}, headers=headers, timeout=timeout)
            if r.status_code != 200: raise RuntimeError(f"HTTP {r.status_code}: {r.text[:300]}")
            try: data = r.json()
            except: data = json.loads(r.text)
            return data
        except Exception as e:
            last=e; time.sleep(0.5*(att+1))
    raise RuntimeError(f"Falha API: {last}")

# -------- parâmetros Radar --------
RADAR_URL = "http://172.31.16.5:8089/radarwebwebservices/Areas/Executivo/Executivo.svc/json/BuscarRelatorioExportacaoAutomatica"
link_a = 'http://172.31.16.5:8089/radarwebwebservices/Areas/Gerenciador/Gerenciador.svc/json/GravarContratoGerenciador'
link_b = 'http://172.31.16.5:8089/radarwebwebservices/Areas/Gerenciador/Empresarial.svc/json/BuscarCondicoesPagamento'
headers={"Content-Type":"application/json","Accept":"application/json"}
login_info_api = {"Base":"LOCAVIA","Usuario":"Igor.costa","Senha":"Rogi!@#"}
CFG_CLIENTES_TXT      = r"C:\WKRadar\BI\Config\Clientes.txt"
CFG_NFS_TXT    = r"C:\WKRadar\BI\Config\NFs.txt"
CFG_TABELAS_TXT = r"C:\WKRadar\BI\Config\Tab_Venda_Serviços.txt"
CFG_CONTRATOS_TXT    = r"C:\WKRadar\BI\Config\ContratosClientes.txt"
CFG_SERVICOS_TXT    = r"C:\WKRadar\BI\Config\Servicos.txt"
CFG_GERENCIAL_TXT    = r"C:\WKRadar\BI\Config\Gerencial.txt"
CFG_CONDPAG_TXT = r"C:\WKRadar\BI\Config\CONDICAO_PAGAMENTO.txt"

@contextmanager
def tqdm_joblib(tqdm_object):
    class _CB(joblib.parallel.BatchCompletionCallBack):
        def __call__(self,*a,**k):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*a,**k)
    old = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = _CB
    try: yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old
        tqdm_object.close()

def fetch_data_from_config(txt: str, date_ref: pd.Timestamp)->pd.DataFrame:
    a = pd.Timestamp(date_ref).replace(day=1)
    b = pd.Timestamp(date_ref)
    if txt not in ['CFG_CLIENTES_TXT', 'CFG_TABELAS_TXT', 'CFG_CONTRATOS_TXT', 'CFG_SERVICOS_TXT', 'CFG_GERENCIAL_TXT', 'CFG_CONDPAG_TXT']:
        cfg = inject_dates(read_config_template(txt), a, b)
    else:
        cfg = read_config_template(txt)
    return radar_post_json(RADAR_URL, login_info_api, cfg)

def fetch_all_api(start=None, end=None, n_jobs=4, timeout=600):
    dates = pd.date_range(start, end, freq="ME")
    with tqdm_joblib(tqdm(desc="API NFs", total=len(dates))):
        nfs = Parallel(n_jobs=n_jobs, backend="threading")(delayed(fetch_data_from_config)(CFG_NFS_TXT,d) for d in dates)
    
    return pd.concat([pd.DataFrame(dfNF) for dfNF in nfs], ignore_index=True)

In [31]:
hoje = pd.Timestamp.today()

# Ano e mês atuais
y = hoje.strftime('%Y')
m = hoje.strftime('%m')

# Próximo mês (corrige automaticamente dezembro → janeiro)
next_month_date = hoje + pd.DateOffset(months=1)
nextMonth = next_month_date.strftime('%m')
nextYear = next_month_date.strftime('%Y')


# Último dia do mês atual
d = str(calendar.monthrange(hoje.year, hoje.month)[1])

# Mês/Ano atual
mesAno = hoje.strftime('%m/%Y')

# Daqui a um ano
umAno = hoje + pd.DateOffset(years=1)
umAno = umAno.strftime('%d/%m/%Y')

In [32]:
nfs = fetch_all_api(pd.Timestamp("2025-01-01"), pd.Timestamp.today().strftime('%Y-%m')+'-'+d, n_jobs=2)
nfs['Vlr Total'] = [float(a) if a != '' else None for a in nfs['Vlr Total'].str.replace('.','').str.replace(',','.')]

API NFs: 100%|██████████| 14/14 [00:25<00:00,  1.85s/it]


In [33]:
nfs['Nr. NFS-e'] = nfs['Nr. NFS-e'].replace('', None)

In [34]:
cols = ['Nome Fantasia Filial', 'Nº NF', 'Data Emissão NF', 'Código Cliente',
       'Nome Cliente']
for col in cols:
    nfs[col] = [a if a != '' else None for a in nfs[col]]
    nfs[col] = nfs[col].ffill()

In [35]:
nfs1 = []
for nf in nfs['Nº NF'].unique():
    temp = nfs[nfs['Nº NF'] == nf]
    temp['Nr. NFS-e'] = temp['Nr. NFS-e'].ffill()
    nfs1.append(temp)
nfs = pd.concat(nfs1, ignore_index=True)
nfs1 = None
nfs['Complement Prod/Serv'] = nfs['Complement Prod/Serv'].str.replace(' ', '')

C:\Users\alexandro.silva\AppData\Local\Temp\9\ipykernel_20604\2329728356.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Nr. NFS-e'] = temp['Nr. NFS-e'].ffill()


In [36]:
cli = pd.DataFrame(fetch_data_from_config(CFG_CLIENTES_TXT,pd.Timestamp("2025-01-01")))
tab = pd.DataFrame(fetch_data_from_config(CFG_TABELAS_TXT,pd.Timestamp("2025-01-01")))
con = pd.DataFrame(fetch_data_from_config(CFG_CONTRATOS_TXT,pd.Timestamp("2025-01-01")))
serv = pd.DataFrame(fetch_data_from_config(CFG_SERVICOS_TXT,pd.Timestamp("2025-01-01")))
ger = pd.DataFrame(fetch_data_from_config(CFG_GERENCIAL_TXT,pd.Timestamp("2025-01-01")))
ger = ger[ger['Situação'] != 'Inativa']

In [37]:
#atualizar para api
at = pd.read_csv('C:/WKRadar/BI/Registros/Atendimentos_Loctr_Valor.csv', encoding='CP1252', sep=';', dtype=str)
at['S.Descrição'] = at['S.Descrição'].astype(str)
at['Emissão'] = pd.to_datetime(at['Emissão'], format='%d/%m/%y')
at = at[at['Emissão'] >= pd.to_datetime('2024-01-01')].reset_index(drop='index')
at['S.Quantidade'] = at['S.Quantidade'].str.replace('.','').str.replace(',','.').str.replace(':','.').astype(float)
at['PLACA'] = at['PLACA'].astype(str)

In [38]:
at = at[(at['Número NF'].isin(nfs['Nr. NFS-e'].to_list())) & (pd.notna(at['S. Código Serviço']))].reset_index(drop=True)

In [39]:
dePara = pd.read_excel('DePara.xlsx')

In [41]:
sleep(5)
for _, NfRow in at[['Número NF', 'Filial']].drop_duplicates().iterrows():
    nf = NfRow['Número NF']
    fAt = NfRow['Filial']
    fNf = 'LOCAVIA' if fAt == '1' else 'LOCTR'
    tempAt = at[(at['Número NF'] == nf) & (at['Filial'] == fAt)].reset_index(drop=True)
    temp = nfs[(nfs['Nr. NFS-e'] == nf) & (nfs['Nome Fantasia Filial'] == fNf)].reset_index(drop=True)
    tempCol = temp[temp['Nome do Prod/Serv'].str.contains('COLETA')].reset_index(drop=True)
    if tempCol.shape[0] > 0:
        x = 0
        for i, row in temp.iterrows():
            if 'COLETA' in str(row['Nome do Prod/Serv']):
                if 'OS:' in row['Complement Prod/Serv'] and 'MTR' in row['Complement Prod/Serv']:
                    nOS = row['Complement Prod/Serv'][row['Complement Prod/Serv'].index('OS:')+3:row['Complement Prod/Serv'].index('MTR')]
                    if nOS in at['Nº Atend.'].to_list():
                        placaNf = row['Nome Conta Gerencial']
                        placaAt = at.loc[at['Nº Atend.'] == nOS]['PLACA'].values[0]
                      
                        if placaAt != 'nan' and pd.notna(placaAt):
                            gerencial = dePara.loc[dePara['PLACAS'] == placaAt]['GERENCIAL'].values[0]
                            codGer  = ger.loc[ger['Descrição'] == gerencial, 'Código'].values[0]
                            if gerencial != placaNf:
                                if x == 0:
                                    sleep(2)
                                    pag.click(x=449, y=66) #clica no filtro 
                                    sleep(1)
                                    pag.click(x=1443, y=828)
                                    sleep(0.5)
                                    pag.click(x=521, y=287) # clica na data inicial
                                    sleep(0.5)
                                    pag.hotkey('ctrl', 'a')
                                    sleep(0.5)
                                    pag.write(row['Data Emissão NF'])
                                    sleep(0.5)
                                    pag.press('tab')
                                    sleep(0.5)
                                    pag.click(x=562, y=527 ) # clica na filial
                                    sleep(0.5)
                                    pag.press(fAt)
                                    sleep(0.5)
                                    pag.press('tab')
                                    sleep(0.5)
                                    pag.click(x=652, y=232) #clica na aba NFe/NFSe
                                    sleep(1)
                                    pag.click(x=617, y=292) #clica no campo para inserir o numero da nota
                                    sleep(1)
                                    pag.write(nf)
                                    sleep(1)
                                    pag.press('tab')
                                    sleep(1)
                                    pag.press('enter')
                                    sleep(1)
                                    pag.click(x=880, y=119) # clica no primeiro lançamento
                                    sleep(1)
                                    pag.press('alt') # vai em arquivo
                                    sleep(1)
                                    pag.press('right') # vai em editar
                                    sleep(1)
                                    pag.press('down')
                                    pag.press('down', 12)  # forma simplificada # vai ate alterar lançamento gerenciais e clica
                                    sleep(1)
                                    pag.press('enter')
                                pag.click(x=817, y=371)
                                sleep(1)
                                [pag.press('up') for a in range(i)]
                                [pag.press('down') for a in range(i)]
                                sleep(1)
                                pag.click(x=703, y=628) # clica no rateio
                                sleep(1)
                                pag.click(x=1639, y=918) # clica no Alterar
                                sleep(1)
                                pag.click(x=952, y=587) # clica sobre a placa
                                sleep(1)                                
                                pag.click(x=989, y=769) # clica sobre a conta
                                sleep(1)
                                pag.hotkey('ctrl', 'a')
                                sleep(1)
                                pag.press('del')
                                sleep(0.5)
                                pag.write(codGer)
                                sleep(0.5)
                                [pag.press('tab') for a in range(2)]
                                sleep(1)
                                pag.press(' ')
                                sleep(0.3)
                                pag.press('enter')
                                sleep(0.5)
                                x += 1
        if x > 0:
            pag.click(x=1770, y=983) # gravar

In [27]:
fNf

'LOCTR'

In [ ]:
# Criar botão de parada
parar_execucao = False
botao_parar = widgets.Button(description="🛑 PARAR", button_style='danger')
output = widgets.Output()

def ao_clicar_parar(b):
    global parar_execucao
    parar_execucao = True
    with output:
        print("🛑 Parando execução...")

botao_parar.on_click(ao_clicar_parar)
display(botao_parar, output)

In [20]:
sleep(5)
pag.position()

Point(x=562, y=527)